# Playground for Riot Watcher

This is a testing notebook where I explore and test out features of `riotwatcher`. 

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import zilean
from zilean.snapshots import SnapShots

import json

api_key = zilean.read_api_key()
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

## Match Timeline Summary

In [3]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_matchid = my_matches[2]
latest_match = watcher.match.by_id(my_region, latest_matchid)
latest_timeline = watcher.match.timeline_by_match(my_region, latest_matchid)